In [ ]:
import uproot

In [ ]:
files = 'root://sci-xrootd.jlab.org//osgpool/eic/ATHENA/RECO/SINGLE/pi+/1GeV/3to50deg/pi+_1GeV_3to50deg.0002.root:events'

In [ ]:
full_events = ur.concatenate(files, ["mcparticles2.psx", "mcparticles2.psy", "mcparticles2.psz", "mcparticles2.pdgID", "mcparticles2.genStatus", "mcparticles2.g4Parent"], library = 'np')